import python, apache beam modules

In [161]:
from datetime import date, datetime
from datetime import datetime, timezone
import json
import apache_beam as beam
from typing import NamedTuple
from apache_beam import coders
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText

Reading the data from public GCS bucket 

In [162]:
pipe = beam.Pipeline()

def print_row(row):
    print(type(row))
# regex = r'^\d+\s\w+?$'
ip = (pipe
      |beam.io.ReadFromText("gs://cloud-samples-data/bigquery/sample-transactions/transactions.csv", skip_header_lines = True)
      |beam.Map(lambda x: x.split(","))
    #   |beam.Map(print)
)

Filtering out the required columns

In [163]:
ip = ip|beam.Map(lambda y: [y[0],y[3]])
# |beam.Map(print)

Find all transactions have a `transaction_amount` greater than `20`

In [164]:
ip = ip|beam.Filter(lambda a: float(a[1])>20)
# |beam.Map(print)

Exclude all transactions made before the year `2010`

In [165]:
ip = ip|beam.Filter(lambda d: datetime.strptime(d[0], "%Y-%m-%d %H:%M:%S UTC").replace(tzinfo=timezone.utc).year>2010)
# |beam.Map(print)

Sum the total by `date`

In [166]:
ip = ip|beam.Map(lambda d: (str(datetime.strptime(d[0], "%Y-%m-%d %H:%M:%S UTC").replace(tzinfo=timezone.utc).date()), float(d[1])))|beam.CombinePerKey(sum)
# |beam.Map(print)

Save the output into `output/results.csv`

In [167]:
ip = ip|beam.Map(lambda row: ', '.join([str(column) for column in row]))

In [168]:
ip|beam.io.WriteToText('output/task1/result', file_name_suffix='.csv', header='date,total_amount' ,shard_name_template='')|beam.Map(print)

<PCollection[[168]: Map(print).None] at 0x7f89b3b09ac0>

Group all transform steps into a single `Composite Transform`

In [169]:
ip = (pipe
      |beam.io.ReadFromText("gs://cloud-samples-data/bigquery/sample-transactions/transactions.csv", skip_header_lines = True)
      |beam.Map(lambda x: x.split(","))
      |beam.Map(lambda y: [y[0],y[3]])
      |beam.Filter(lambda a: float(a[1])>20)
      |beam.Filter(lambda d: datetime.strptime(d[0], "%Y-%m-%d %H:%M:%S UTC").replace(tzinfo=timezone.utc).year>2010)
      |beam.Map(lambda d: (str(datetime.strptime(d[0], "%Y-%m-%d %H:%M:%S UTC").replace(tzinfo=timezone.utc).date()), float(d[1])))
      |beam.CombinePerKey(sum)
      |beam.Map(lambda row: ', '.join([str(column) for column in row]))
      |beam.io.WriteToText('output/task2/result', file_name_suffix='.csv', header='date,total_amount' ,shard_name_template='')
      |beam.Map(print)
)

In [170]:
pipe.run()

output/task1/result.csv
output/task2/result.csv
